In [1]:
#%pip install langchain chromadb sentence-transformers ollama pandas openpyxl
#%pip install rank_bm25

In [2]:
# Import necessary libraries
import pandas as pd
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS
import pickle
import os

/Users/mbodla/Documents/Webinar Sesssions - AI/AIWebinar/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:

# === Step 1: Load Excel and Convert to LangChain Documents (Run Once) ===
df = pd.read_excel("CPQCOE-AutomatedTestCases-Scope.xlsx")
df.fillna("", inplace=True)

def build_context(row):
    return f"""
TC_ID: {row['TC_ID']}
Automated Test script Name: {row['Automated Test script Name']}
Quote Creation Source: {row['Quote Creation Source']}
Copy Quote: {row['Copy Quote']}
LOB: {row['LOB']}
Sales Channel: {row['Sales Channel']}
Transaction Type: {row['TransactionType']}
OperationType: {row['OperationType']}
Product Configuration: {row['Product Configuration']}
Store Acceptance Type: {row['Store Acceptance Type']}
Customer Type: {row['Customer Type']}
Comments: {row['Comments']}
"""

documents = [Document(page_content=build_context(row)) for _, row in df.iterrows()]


/var/folders/sx/jdc7r4cj4mg159qbrg9k88g80000gq/T/ipykernel_29279/2625694685.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("", inplace=True)


In [4]:


from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.retrievers import BM25Retriever

# === Step 2: Create or Load Vector Store (Only Once) ===
# Chunking the documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunked_documents = text_splitter.split_documents(documents)

# Create embeddings and vector store based on chunks
embedding_model = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')
faiss_index_path = "faiss_index.pkl"

if os.path.exists(faiss_index_path):
    with open(faiss_index_path, "rb") as f:
        vectorstore = pickle.load(f)
else:
    vectorstore = FAISS.from_documents(chunked_documents, embedding_model)
    with open(faiss_index_path, "wb") as f:
        pickle.dump(vectorstore, f)

# Create vector store retriever
vector_retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

# Create BM25 retriever
keyword_retriever = BM25Retriever.from_documents(chunked_documents)


#Ensemble Retriever
retriever = EnsembleRetriever(retrievers=[vector_retriever,
                                                   keyword_retriever],
                                       weights=[0.5, 0.5])


/var/folders/sx/jdc7r4cj4mg159qbrg9k88g80000gq/T/ipykernel_29279/602267539.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')
/Users/mbodla/Documents/Webinar Sesssions - AI/AIWebinar/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
# === Step 3: Define Mistral via Ollama + Prompt ===
llm = Ollama(model="mistral")

qa_prompt = PromptTemplate(
    input_variables=["context", "query"],
    template="""
You are a QA assistant helping map keyword-based queries to test cases.

Given the following keywords:
"{query}"

And the following test case descriptions:
{context}

Return a list of the most relevant TC_IDs that match the query.
For each TC_ID, include a short explanation why it's relevant.
Only respond with actual TC_IDs from the input.
at the end of the response only print the TC_IDs, separated by commas.
"""
)

qa_chain = LLMChain(llm=llm, prompt=qa_prompt)


In [ ]:
# Extract only TC_ID's from the response
def extract_tc_ids(response):
    """
    Extracts TC_IDs from the given response string.

    Args:
        response (str): The response string containing TC_IDs and explanations.

    Returns:
        list: A list of extracted TC_IDs.
    """
    return [line.split(":")[1].strip() for line in response.split("\n") if line.startswith("TC_ID")]

In [20]:
# === Step 4: User Query Loop ===

def Conversation():
    print("Chatbot: Hello! I'm QA Agent. Please enter your keywords to retrieve the corresponding TC_ID's.\nType 'bye' to exit.")
    print("\n✅ Ready to search test cases! Type 'exit' to stop.\n")
    while True:
        user_input = input("🔍 Enter comma-separated keywords: ")
        if user_input.lower() in ["exit", "quit"]:
            break

        query = " ".join(user_input.split(","))
        print(f"\n🔑 Keywords entered: {query}")
        
        matched_docs = retriever.get_relevant_documents(query)
        context = "\n\n".join([doc.page_content for doc in matched_docs])

        final_response = qa_chain.run(context=context, query=query)

        print("\n📄 === Retrieved Test Case IDs ===")
        print(final_response)
        print("\n" + "-"*50 + "\n")

In [8]:
Conversation()

Chatbot: Hello! I'm OM-Bot. Please enter your Order Number along with your query.
Type 'bye' to exit.

✅ Ready to search test cases! Type 'exit' to stop.


📄 === Retrieved Test Case IDs ===
1. TC_ID: CPQ-SILO-RT-20, CPQ_RT_20_SILO_CLOUD_PublicSectorPaymentTerms - This test case is relevant because it contains the keyword "Publicsector" and it's related to the Cloud product line (LOB), which seems to be a common factor across many of your test cases.

  2. TC_ID: CPQ-SILO-RT-DD11, CPQ_RT_DD11_20B_SILO_CLOUD_PublicSectorPaymentTerms_DE - This test case is relevant because it also contains the keyword "Publicsector" and it's related to the Cloud product line (LOB) for a German (DE) context, which might be important depending on your requirements.

  3. TC_ID: CPQ-SILO-RT-DD10, CPQ_RT_DD10_20A_SILO_CLOUD_PublicSectorPaymentTerms_CA - This test case is relevant because it contains the keyword "Publicsector" and it's related to the Cloud product line (LOB) for a Canadian (CA) context.

  4. 

In [10]:
Conversation()

Chatbot: Hello! I'm OM-Bot. Please enter your Order Number along with your query.
Type 'bye' to exit.

✅ Ready to search test cases! Type 'exit' to stop.


🔑 Keywords entered: ucm

📄 === Retrieved Test Case IDs ===
1. CPQ-SILO-RT-UCM4: This test case is relevant because it involves the UCM product configuration and Direct Onboarding in the Cloud sales channel, which are keywords present in the query.

  2. CPQ-SILO-RT-UCM3: This test case also involves the UCM product configuration and Direct Onboarding in the Cloud sales channel, making it relevant to the query.

  3. CPQ-SILO-RT-UCM2: Although this test case does not explicitly mention "UCM4", it involves the same "UCM" product and Direct Onboarding in the Cloud sales channel, which makes it related to the query.

  4. CPQ-SILO-RT-CTP-002: While this test case is about UCM-PAYG rather than UCM4, it does involve the UCM product and Direct Onboarding in the Cloud sales channel, making it relevant to the query.

------------------------

In [22]:
Conversation()

Chatbot: Hello! I'm QA Agent. Please enter your keywords to retrieve the corresponding TC_ID's.
Type 'bye' to exit.

✅ Ready to search test cases! Type 'exit' to stop.


🔑 Keywords entered: Professional Services

📄 === Retrieved Test Case IDs ===
 Based on the keyword "Professional Services", the following test cases are most relevant:

1. TC_ID: CPQ-SILO-RT-15 - This test case includes the "Product Configuration: Professional Services" and is associated with the "LOB: Cloud".
2. TC_ID: CPQ-SILO-RT-CTP-QA2 - This test case also includes the "Product Configuration: Professional Services" and is associated with the "LOB: Cloud".
3. TC_ID: CPQ-SILO-RT-31 - While this test case specifies a specific product type ("Consulting Product") within professional services, it still pertains to professional services as a whole.
4. TC_ID: CPQ-SILO-RT-23 - This test case includes the "Product Configuration: Professional Services" and is associated with the "LOB: Cloud".
5. TC_ID: CPQ-SILO-RT-35 - Altho

In [ ]:
import requests

# Create Jenkins agent to trigger a build with the TC_ID's generated
# from the conversation
def trigger_jenkins_build(tc_ids):
    """
    Triggers a Jenkins build with the given TC_IDs.

    Args:
        tc_ids (list): List of TC_IDs to pass to the Jenkins job.
    """
    if not tc_ids:
        print("No TC_IDs provided to trigger the Jenkins build.")
        return

    jenkins_url = "http://phoenix172606.ad1.fusionappsdphx1.oraclevcn.com:8080"
    jenkins_user = "admin"
    jenkins_token = "jenkins_token"
    params = {
        "TC_IDs": ",".join(tc_ids)  # Pass TC_IDs as a comma-separated string
    }

    try:
        response = requests.post(jenkins_url, auth=(jenkins_user, jenkins_token), params=params)
        if response.status_code == 201:
            print("Jenkins build triggered successfully!")
        else:
            print(f"Failed to trigger Jenkins build. Status code: {response.status_code}, Response: {response.text}")
    except Exception as e:
        print(f"An error occurred while triggering the Jenkins build: {e}")
# def trigger_jenkins_build(tc_ids):
#     # Code to trigger Jenkins build with the TC_IDs
#     pass
# Conversation()
